In [3]:
from sympy.parsing.sympy_parser import parse_expr

In [2]:
parse_expr("0.3x + 3y", evaluate=True, local_dict={"x": 10, "y": 3, "z": 5}, transformations="all")

12

In [3]:
from sympy import parse_expr, postorder_traversal
from expressiontree import *
import re



contains_operators("x_y")

False

In [2]:
from expression import Expression
expr = Expression(portfolio_assets=["AAPL"], str_expression="0.1 + _AAPL(return, 3d)")

complex_f = expr.evaluate("_AAPL(return, 5d) + _transpose(_AAPL(return, 10d, 1d) + _AAPL(volume, 10d, 1d))")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
0
[*********************100%***********************]  1 of 1 completed
1
0.04692692172716728 + _transpose(AAPLreturn10d1d_0 + AAPLvolume10d1d_1)


In [4]:
expr.get_function_arguments("_transpose(AAPLreturn10d1d_0 + AAPLvolume10d1d_1)")

['AAPLreturn10d1d_0+AAPLvolume10d1d_1']

In [23]:
from sympy import Matrix, parse_expr

m = Matrix([1, 2, 3])

parse_expr("m/m", local_dict={"m": m}, evaluate=True)

TypeError: unsupported operand type(s) for /: 'One' and 'MutableDenseMatrix'

In [21]:
x = Matrix([1, 2, 3, 4])
x.reshape(2, 2) + x.reshape(2, 2)

Matrix([
[2, 4],
[6, 8]])

In [4]:
from numericals import Numerical

numerical = Numerical()

numerical.summarize_historical_data("AAPL")

[*********************100%***********************]  1 of 1 completed


{'period': '100d',
 'interval': '1d',
 'time_zone': None,
 'average_open': 150.29220054626464,
 'max_open': 173.0500030517578,
 'min_open': 126.01000213623047,
 'stdev_open': 12.932928708865637}

In [4]:
from expression import Expression

expr = Expression(portfolio_assets=["AAPL"], str_expression="3x^2 + average(AAPL_return, 10d, 1m) * average(what, 5d, 1s)", variable_dict={"x": 10})
expr.evaluate("3x^2 + _AAPL(return, 10d, 1d)")

[*********************100%***********************]  1 of 1 completed
{'AAPLreturn10d1d_0': Matrix([
[-0.000530401841079775],
[ -0.00619142549107132],
[ -0.00646725620071475],
[ -0.00991345347068961],
[   0.0469269474632046],
[-0.000403367369252505],
[ -0.00997112231930308],
[   0.0104208753759483],
[  0.00109475392335234],
[ -0.00541756894991818]])}


'3x^2 + AAPLreturn10d1d_0'

In [10]:
from expressiontree import *

# solve_expression("3x^2 + AAPLreturn10d1d_0", variable_dict={"AAPLreturn10d1d_0": 10}  )
get_ordered_operations("3x^2 + AAPLreturn10d1d_0")

['x**2', '3*x**2', 'AAPLreturn10d1d_0 + 3*x**2']

In [6]:
from expressiontree import *

solve_expression("x * 500", {"x": Matrix([1, 2, 3]), "y": Matrix([2, 3, 4])})

Matrix([
[ 500],
[1000],
[1500]])

In [51]:
# solve_expression("x + y + 100", {"x": 3, "y": 10})
get_ordered_operations("x + (y + 100)")

['x + y + 100']

In [46]:
from sympy import postorder_traversal

for i in postorder_traversal(parse_expr("x /y/ 100", evaluate=False, transformations="all")):
    print(i)

x
y
-1
1/y
100
-1
1/100
x/(y*100)


In [32]:
# from expressiontree import *
from sympy import Matrix
import re

def make_matrix_aware(operand1, operand2, operator, variable_dict):
    # operands in variable_dict are matrics. We are checking to determine how to process
    # when one is a matrix and another one not, or when both are or are not matrics
    # We are parsing expression containing matrices slighly different from how it is done
    # in sympy which doesn't allow operations such as "3 + Matrix([1, 2, 3])". We want to
    # make such kind of operation possible in our app though, so we are extending on what
    # is offered by sympy
    if(operand1 in variable_dict and operand2 in variable_dict):
        return "{op_1}{op}{op_2}".format(op_1=operand1, op=operator, op_2=operand2)
    elif(operand1 not in variable_dict and operand2 not in variable_dict):
        return "{op_1}{op}{op_2}".format(op_1=operand1, op=operator, op_2=operand2)
    elif(operator != "+" and operator != "-"):
        return "{op_1}{op}{op_2}".format(op_1=operand1, op=operator, op_2=operand2)
    elif(operand1 in variable_dict and operand2 not in variable_dict):
        # operand1 is a matrix while operand2 is not
        operand1 = variable_dict[operand1]
        matrix_operand_size = operand1.shape[0]*operand1.shape[1]
        transformed_operand2 = Matrix([operand2 for i in range(matrix_operand_size)])
        transformed_operand2 = transformed_operand2.reshape(operand1.shape[0], operand1.shape[1])
        return "{op_1}{op}{op_2}".format(op_1=operand1, op=operator, op_2=transformed_operand2)
    elif(operand2 in variable_dict and operand1 not in variable_dict):
        # operand2 is a matrix while operand1 is not
        operand2 = variable_dict[operand2]
        matrix_operand_size = operand2.shape[0]*operand2.shape[1]
        transformed_operand1 =  Matrix([operand1 for i in range(matrix_operand_size)])
        transformed_operand1 = transformed_operand1.reshape(operand2.shape[0], operand2.shape[1]) 
        return "{op_1}{op}{op_2}".format(op_1=transformed_operand1, op=operator, op_2=operand2)

In [33]:
make_matrix_aware(operand1="y", operand2="8", operator="-", variable_dict={"y": Matrix([1, 2, 3])})

'Matrix([[1], [2], [3]])-Matrix([[8], [8], [8]])'

In [26]:
from sympy import *

Matrix([1, 2, 2]) + 3

TypeError: cannot add <class 'sympy.matrices.dense.MutableDenseMatrix'> and <class 'int'>

In [8]:
def get_innermost_functions(str_expression, arguments_count):

    # The ",\s*\w+" extracts a single argument. We want to be flexible with the
    # number of arguments we can extract. For instance, if we wnat to extract
    # an inner function that accepts two arguments, the regex for that will
    # be r"\w+\(\w+\s*,\s*\w+\)". That is what we are doing here
    r = "".join([",\s*\w+\s*" for i in range(arguments_count-1)]) + r"\)"
    regex = r"\w+\(\s*\w+\s*" + r

    return re.findall(regex, str_expression)

what = get_innermost_functions(str_, 3)
what

NameError: name 'str_' is not defined

In [2]:
my_arr = ["AAPL", "What"]
what = ["_"+i for i in my_arr]
what

['_AAPL', '_What']

In [1]:
from expression import Expression
strr = "_AAPL(return, 1d) + _AAPL(return, 10d, 1d)"
expr = Expression(str_expression=strr, portfolio_assets=["AAPL", "MSFT"])
expr.evaluate()

[*********************100%***********************]  1 of 1 completed


'0.006432995866882063 + _AAPL(return, 10d, 1d)'

In [12]:
simple_functions_used = expr.get_simple_functions()
simple_functions_used

{'_AAPL': ['_AAPL(return, 1d)']}

In [1]:
from numericals import Numerical

numerical = Numerical()
data = numerical.get_historical_data("AAPL", period="2d")
data

period:  3d
[*********************100%***********************]  1 of 1 completed


,open,high,low,close,adj_close,volume,return
Date,,,,,,,
2023-05-09,173.050003,173.539993,171.600006,171.770004,171.770004,45285300,-0.009971
2023-05-10,173.020004,174.029999,172.320007,173.670105,173.670105,21280764,0.011062


In [7]:
data = data.iloc[1:, :]
data

,open,high,low,close,adj_close,volume,return
Date,,,,,,,
2023-05-10,173.020004,174.029999,172.320007,173.744995,173.744995,21196692,0.011498


In [14]:

f_expression_results = {}
for f in simple_functions_used:
    for expression in simple_functions_used[f]:
        
        val = evaluate_simple_function(function_=f, expression=expression)
        f_expression_results[expression] = val

for key in f_expression_results:
    str_expression = str_expression.replace(key, str(f_expression_results[key]))

[*********************100%***********************]  1 of 1 completed


NameError: name 'str_expression' is not defined

In [6]:
import yfinance as yf

yf.download("AAPL", period="10d")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-27,165.190002,168.559998,165.190002,168.410004,168.410004,64902300
2023-04-28,168.490005,169.850006,167.880005,169.679993,169.679993,55209200
2023-05-01,169.279999,170.449997,168.639999,169.589996,169.589996,52472900
2023-05-02,170.089996,170.350006,167.539993,168.539993,168.539993,48425700
2023-05-03,169.500000,170.919998,167.160004,167.449997,167.449997,65136000
2023-05-04,164.889999,167.039993,164.309998,165.789993,165.789993,81235400
2023-05-05,170.979996,174.300003,170.759995,173.570007,173.570007,113316400
2023-05-08,172.479996,173.850006,172.110001,173.500000,173.500000,55962800
2023-05-09,173.050003,173.539993,171.600006,171.770004,171.770004,45285300
